<style>
body {
    max-width: 900px;
    margin: 40px auto;
    padding: 0 20px;
    font-family: "Georgia", serif;
    line-height: 1.6;
}
</style>

<div style="text-align: center; padding: 60px 60px">
  <h1 style="font-weight: bold; font-size: 3.1em">
    ENCONTRAR EL MEJOR ARIMA PARA LA SERIE DIFERENCIADA
  </h1>
</div>

No correr este archivo, tarda mucho, mejor importar los resultados.

Se cree que el mejor modelo, que es un modelo admisible, está cuando solo se varía la parte no estacional. También, que el $\text{AIC}$, dependiendo el número de parámetros, para esta serie de tiempo, forma un valle, y el mejor está entre $12$ y $18$.

**RESULTADOS**: 
- El modelo con menor $\text{AIC}$ es el $\text{ARIMA}(0,0,15) \times (0,1,0)$,  Con un $\text{AIC} = 1370.8057965209173$.
- `aic_values-1.csv`
- `AIC_3D_plot_1.html`.

# **CARGAR LOS DATOS**

In [1]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

data=pd.read_csv('MXN00021035.csv')

pre=data.iloc[:,6]  # Precipitacion, es la columna 5
date=data.iloc[:,5] # Date, es la columna 6
date = date.astype(str).str.replace(r'(\d{4})(\d{2})', r'\1/\2', regex=True)    # La fecha está como 195210 y la pasamos a 1952/10 
date = pd.to_datetime(date, format='%Y/%m')                                     # Lo convertimos en fecha
pre = pd.Series(pre.values, index=date)                                         # Creamos una Serie

# Partir la serie para train y test
pre_total = pre.copy()          # Copia de la serie original

# Todas hasta los ultimos 12 meses
pre = pre_total[:-12]           # Entrenamiento: todos menos los últimos 12 meses

from scipy.stats import zscore

# Estandarizar la serie original
zpre = zscore(pre)

# **BUSCAMOS EL MODELO CON MENOR $\text{AIC}$**

Se fue probando varias veces, y se buscaba ir encontrando el mejor $\text{AIC}$  en saltos, porque hacer todos los modelos tarda mucho tiempo. 

Para hacer mejor la legibilidad de la notebook, se optó por remover todo el proceso de pruebas para encontrar los $\text{AIC}$ de distintos modelos. Sin embargo, se pueden encontrar los resultados en `aic_values-1.csv`.

In [ ]:
def sarima(p, d, q, P, D, Q, s):    # La vamos a usar para calcular el AIC
    return SARIMAX(zpre, order=(p, d, q), seasonal_order=(P, D, Q, s)).fit(disp=0).aic    

In [ ]:
aics = []   # Lista para guardar los AICs

for p in range(, ):     # Para el primer  p
    for q in range(, ): # Para el segundo  q
        aic = sarima(p, 0, q, 0, 1, 0, 12)  # Ajustamos el modelo SARIMA y guardamos el AIC
        aics.append((p, q, aic))

In [104]:
aic = sarima(10, 0, 6, 0, 1, 0, 12)
aics.append((10, 6, aic))  # Guardamos el AIC del modelo SARIMA(10,0,6)(0,1,0,12)

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [107]:
sorted(aics, key=lambda x: x[2])

[(0, 15, 1370.8057965209173),
 (0, 12, 1370.924382974263),
 (3, 12, 1372.2524267237013),
 (0, 16, 1372.5235049900543),
 (1, 15, 1372.5974937313972),
 (1, 12, 1372.686044503483),
 (0, 13, 1372.7768523911463),
 (3, 13, 1372.8730211579154),
 (4, 12, 1373.4136239901386),
 (1, 16, 1373.5915299075373),
 (1, 13, 1373.7595406046005),
 (3, 14, 1374.0059992042613),
 (4, 13, 1374.118051844699),
 (5, 12, 1374.3444564742897),
 (0, 14, 1374.440877112027),
 (2, 12, 1374.4930635003877),
 (2, 12, 1374.4930635003877),
 (0, 17, 1374.4949956797102),
 (2, 15, 1374.6209289634921),
 (2, 15, 1374.6209289634921),
 (1, 17, 1374.9667226281185),
 (4, 14, 1375.0815279093479),
 (2, 13, 1375.156046502),
 (2, 13, 1375.156046502),
 (1, 14, 1375.2555648291286),
 (5, 13, 1375.4319196086317),
 (6, 12, 1375.942622502207),
 (6, 12, 1375.942622502207),
 (2, 16, 1375.9650754134357),
 (2, 16, 1375.9650754134357),
 (3, 15, 1376.3703456954474),
 (7, 12, 1376.4595793206606),
 (2, 14, 1376.6336635433652),
 (2, 14, 1376.6336635433

El modelo con menor $\text{AIC}$ es el $\text{ARIMA}(0,0,15) \times (0,1,0)$,  Con un $\text{AIC} = 1370.8057965209173$.

In [ ]:
# Guardar los valores AIC en un archivo CSV
aic_df = pd.DataFrame(aics, columns=['p', 'q', 'AIC'])
aic_df.to_csv('aic_values-1.csv', index=False)

# **VISUALIZACIÓN**

Esta sección si se puede correr sin problemas.

In [114]:
import plotly.graph_objects as go
import numpy as np
from scipy.interpolate import griddata

In [115]:
acis = pd.read_csv('aic_values-1.csv')

In [116]:
# Separar en listas
p_values = [item[0] for item in aics]
q_values = [item[1] for item in aics]
aic_values = [item[2] for item in aics]

# Crear una malla regular
p_lin = np.linspace(min(p_values), max(p_values), 50)
q_lin = np.linspace(min(q_values), max(q_values), 50)
P, Q = np.meshgrid(p_lin, q_lin)

# Interpolar AIC en la malla
AIC_grid = griddata(
    points=(p_values, q_values),
    values=aic_values,
    xi=(P, Q),
    method='nearest'
)

In [117]:
# Crear figura 3D en Plotly
fig = go.Figure()

# Superficie
fig.add_trace(go.Surface(
    x=P, y=Q, z=AIC_grid,
    colorscale='Blues_r',
    opacity=0.7,
    colorbar=dict(title='AIC')
))

# Puntos originales
fig.add_trace(go.Scatter3d(
    x=p_values,
    y=q_values,
    z=aic_values,
    mode='markers',
    marker=dict(size=5, color=aic_values, colorscale='Blues_r'),
    name='Puntos'
))

# Layout
fig.update_layout(
    scene=dict(
        xaxis_title='p',
        yaxis_title='q',
        zaxis_title='AIC'
    ),
    title='AIC para diferentes combinaciones de ARIMA(p,0,q)X(0,1,0,12)',
    width=900,
    height=700
)

# Guardar como HTML
fig.write_html("AIC_3D_plot-1.html")
fig.show()

# **VERIFICACIÓN DE SUPUESTOS**

In [2]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from verificacion_de_supuestos import * # Una funcion que hice para verificar si los parametros son estacionarios/invertibles
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import ttest_1samp
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.tools.tools import add_constant
from scipy.stats import jarque_bera
from statsmodels.stats.diagnostic import lilliefors

In [3]:
modelo=SARIMAX(zpre,
               order=(0,0,15),
               seasonal_order=(0,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(0, 0, 15)x(0, 1, [], 12)   Log Likelihood                -669.403
Date:                              Sat, 26 Apr 2025   AIC                           1370.806
Time:                                      21:48:23   BIC                           1442.388
Sample:                                           0   HQIC                          1398.575
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1367      0.037      3.666      0.000       0.064       0.210
ma.L2          0.0006      0.041      0.014      0.988      -0.080       0.081
ma.L3          0.0767      0.041      1.866      0.062      -0.004       0.157
ma.L4         -0.0764      0.040     -1.892      0.058      -0.155       0.003
ma.L5         -0.1454      0.038     -3.816      0.000      -0.220      -0.071
ma.L6         -0.1591      0.041     -3.922      0.000      -0.239      -0.080
ma.L7         -0.0977      0.038     -2.566      0.010      -0.172      -0.023
ma.L8         -0.0202      0.036     -0.554      0.580      -0.091       0.051
ma.L9          0.0189      0.035      0.543      0.587      -0.049       0.087
ma.L10         0.0721      0.034      2.104      0.035       0.005       0.139
ma.L11         0.1653      0.035      4.718      0.000       0.097       0.234
ma.L12        -0.7787      0.036    -21.409      0.000      -0.850      -0.707
ma.L13        -0.0483      0.041     -1.178      0.239      -0.129       0.032
ma.L14        -0.0184      0.039     -0.469      0.639      -0.095       0.059
ma.L15        -0.1024      0.043     -2.372      0.018      -0.187      -0.018
sigma2         0.4461      0.021     21.164      0.000       0.405       0.487
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):               360.03
Prob(Q):                              0.94   Prob(JB):                         0.00
Heteroskedasticity (H):               0.80   Skew:                             0.86
Prob(H) (two-sided):                  0.10   Kurtosis:                         6.22
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [4]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {}

El grado del polinomio es: 0

Raíces del polinomio característico: []

Módulo de las raíces: []

¿Las raíces están fuera del círculo unitario?  True

El modelo es estacionario

:)


In [5]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p1': 0.13670240371999162, 'p2': 0.0005971888480305956, 'p3': 0.07670095541918637, 'p4': -0.07637154759112381, 'p5': -0.14537162831990777, 'p6': -0.1591494261114962, 'p7': -0.09765166095390075, 'p8': -0.020150469201547817, 'p9': 0.018894086400221687, 'p10': 0.07211092136133603, 'p11': 0.1652919906997901, 'p12': -0.7787367794635099, 'p13': -0.04830339922205925, 'p14': -0.018407433802944403, 'p15': -0.10242070791381097}

El grado del polinomio es: 15

Raíces del polinomio característico: [ 0.8140251 +1.79974151j  0.8140251 -1.79974151j -2.00260148+0.j
  0.95007946+0.50126699j  0.95007946-0.50126699j  1.00203779+0.j
  0.51057431+0.86905551j  0.51057431-0.86905551j  0.00842192+1.0215353j
  0.00842192-1.0215353j  -0.50272798+0.87137081j -0.50272798-0.87137081j
 -1.00175188+0.j         -0.8690769 +0.50012473j -0.8690769 -0.50012473j]

Módulo de las raíces: [1.97527375 1.97527375 2.00260148 1.07420649 1.07420649 1.00203779
 1.00794028 1.00794028 1.021

In [6]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.269005,NaN
2,0.273263,NaN
3,0.399129,NaN
4,0.417800,NaN
5,1.551264,NaN
6,1.980990,NaN
7,2.978212,NaN
8,3.536694,NaN
9,4.544775,NaN
10,4.576955,NaN


Todos los residuos son independientes.

In [7]:
modelo.resid.mean()

-0.045524455453858546

Los residuos son un poco mejores que el modelo 14.

In [8]:
ttest_1samp(modelo.resid, 0)

TtestResult(statistic=-1.7052893997409186, pvalue=0.08861172351622751, df=659)

La media de los residuos es cero.

In [9]:
het_breuschpagan(modelo.resid, add_constant(np.arange(len(modelo.resid))))

(2.246413161692278, 0.1339245098780731, 2.247254731819336, 0.13433127866106115)

Los residuos son constantes.

In [10]:
print(jarque_bera(modelo.resid))
print(lilliefors(modelo.resid))

SignificanceResult(statistic=399.6392527449315, pvalue=1.6574441016528617e-87)
(0.11286846354370039, 0.0009999999999998899)


No se distribuye normal.

In [11]:
normal_relajacion(modelo.resid)

74.55% de los residuos están dentro de ±1σ (esperado ≈ 68%)
94.39% de los residuos están dentro de ±2σ (esperado ≈ 95%)
98.64% de los residuos están dentro de ±3σ (esperado ≈ 99.7%)


Puede ser una normal.